# Practice 2



As always we start with uploading needed libraries

In [1]:
from os import path
import mujoco
import mujoco_viewer
import numpy as np

Constant variables such as simulation time, step numbers, model name, and MuJoCo entities: model and data

In [2]:
SIMEND = 10
STEP_NUM = 1000
MODEL_NAME = 'open_chain_robot.xml'

ROOT = path.abspath('')
FOLDER_PATH = path.join(ROOT, '')
MODEL_PATH = path.join(FOLDER_PATH, MODEL_NAME)

model = mujoco.MjModel.from_xml_path(MODEL_PATH)
data = mujoco.MjData(model)

Let us make servo control. 

Take a look at container `actuator` at xml of the model. The the first actuator (id is 0) is a motor, which means it is torque controlled. 

In [3]:
def set_position_servo(mjmodel, actuator_no, kp):
    mjmodel.actuator_gainprm[actuator_no, 0] = kp
    mjmodel.actuator_biasprm[actuator_no, 1] = -kp


def set_velocity_servo(mjmodel, actuator_no, kv):
    mjmodel.actuator_gainprm[actuator_no, 0] = kv
    mjmodel.actuator_biasprm[actuator_no, 2] = -kv


def controller(mjmodel, mjdata):

    # position control; position/velocity servo
    set_position_servo(mjmodel, 1, 100)
    set_velocity_servo(mjmodel, 2, 10)
    mjdata.ctrl[1] = np.pi/4  # position

    set_position_servo(mjmodel, 4, 100)
    set_velocity_servo(mjmodel, 5, 10)
    mjdata.ctrl[4] = np.pi/2  # position

Viewer and simulation

In [4]:
viewer = mujoco_viewer.MujocoViewer(model, data,
                                    title="2R_robot",
                                    width=1920,
                                    height=1080)

mujoco.set_mjcb_control(controller)

for i in range(STEP_NUM):
    if viewer.is_alive:
        time_prev = data.time
        while (data.time - time_prev < 1.0/60.0):
            mujoco.mj_step(model, data)

        if (data.time >= SIMEND):
            break
        viewer.render()
    else:
        break

viewer.close()   

Let us change controller 

In [5]:
def set_torque_servo(actuator_no, flag):
    if (flag == 0):
        model.actuator_gainprm[actuator_no, 0] = 0
    else:
        model.actuator_gainprm[actuator_no, 0] = 1


def controller(model, data):

    # torque control
    set_torque_servo(0, 1)
    data.ctrl[0] = 100*(np.pi/2-data.qpos[0]) + 10*(0.5-data.qvel[0])
    data.ctrl[3] = 100*(np.pi/2-data.qpos[1]) + 10*(0.5-data.qvel[1])

Sim

In [6]:
viewer = mujoco_viewer.MujocoViewer(model, data,
                                    title="2R_robot",
                                    width=1920,
                                    height=1080)

mujoco.set_mjcb_control(controller)

for i in range(STEP_NUM):
    if viewer.is_alive:
        time_prev = data.time
        while (data.time - time_prev < 1.0/60.0):
            mujoco.mj_step(model, data)

        if (data.time >= SIMEND):
            break
        # mujoco.mj_step(model, data)
        viewer.render()
    else:
        break

viewer.close()    